In [1]:
import pandas as pd
import numpy as np
import pickle as p

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.INFO)
#tf.enable_eager_execution()
from sklearn.model_selection import train_test_split

from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

Compile the training data

In [2]:
meta_data = pd.read_csv('/home/ec2-user/training_data_meta.csv')

#Join cleaned class data with training metadata

class_data = pd.read_csv('/home/ec2-user/stanford_labels_cleaned.csv')

meta_data = pd.merge(meta_data, class_data, on = 'class', how = 'left')

meta_data = meta_data.loc[meta_data['Body Type'].isin(['Coupe', 'Sedan'])].copy()

meta_data['is_sedan_target'] = (meta_data['Body Type'] == 'Sedan').astype(int)

image_dict = p.load(open('/home/ec2-user/scaled_bounded_grayscale_dict.p', 'rb'))

training_data = []
for i in meta_data[['is_sedan_target', 'fname']].iterrows():
    row = [i[1]['is_sedan_target']]
    row.extend(image_dict[i[1]['fname']].flatten())
    training_data.append(row)

training_data = np.array(training_data).astype(float)

Prep the data

In [3]:
X_train, X_test, y_train, y_test = train_test_split(training_data[:,1:], 
                                                    training_data[:,0], 
                                                    test_size=0.30, 
                                                    random_state=420)

y_train = y_train.astype('int32')
y_test = y_test.astype('int32')

negatives = []
for ind, elem in enumerate(y_train):
    if elem == 0:
        negatives.append(ind)
np.random.shuffle(negatives)

neg_dupe_target = 2 * y_train.sum() - y_train.shape[0]

X_train = np.concatenate([X_train, X_train[negatives[0:neg_dupe_target],:]],
                         axis = 0
                        )
y_train = np.concatenate([y_train, y_train[negatives[0:neg_dupe_target]]],
                         axis = 0
                        )

y_train = np.concatenate([1 - y_train.reshape(-1,1), y_train.reshape(-1,1)], axis = 1)
y_test = np.concatenate([1 - y_test.reshape(-1,1), y_test.reshape(-1,1)], axis = 1)

Define the model

In [4]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    if type(features) is dict:
        features = features['input']
    #Define model architecture
    
    # Input Layer
    # Reshape X to 4-D tensor: [batch_size, width, height, channels]
    input_layer = tf.reshape(features, [-1, 200, 200, 1])

    # Convolutional Layer #1
    # Computes 32 features using a 3x3 filter with ReLU activation.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 200, 200, 1]
    # Output Tensor Shape: [batch_size, 200, 200, 32]
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=16,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #1
    # First max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 200, 200, 32]
    # Output Tensor Shape: [batch_size, 100, 100, 32]
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2
    # Computes 64 features using a 3x3 filter.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 100, 100, 32]
    # Output Tensor Shape: [batch_size, 100, 100, 64]
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #2
    # Second max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 100, 100, 64]
    # Output Tensor Shape: [batch_size, 50, 50, 64]
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    
#     conv3 = tf.layers.conv2d(
#       inputs=pool2,
#       filters=32,
#       kernel_size=[3, 3],
#       padding="same",
#       activation=tf.nn.relu)

#     # Pooling Layer #3
#     # Second max pooling layer with a 2x2 filter and stride of 2
#     # Input Tensor Shape: [batch_size, 50, 50, 64]
#     # Output Tensor Shape: [batch_size, 25, 25, 64]
#     pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)

    # Flatten tensor into a batch of vectors
    # Input Tensor Shape: [batch_size, 25, 25, 64]
    # Output Tensor Shape: [batch_size, 25 * 25 * 64]
    pool3_flat = tf.reshape(pool2, [-1, 50 * 50 * 32])

    # Dense Layer
    # Densely connected layer with 256 neurons
    # Input Tensor Shape: [batch_size, 25 * 25 * 64]
    # Output Tensor Shape: [batch_size, 256]
    dense = tf.layers.dense(inputs=pool3_flat, units=64, activation=tf.nn.relu)

    # Add dropout operation; 0.6 probability that element will be kept
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.5, training=(mode == tf.estimator.ModeKeys.TRAIN))

    # Logits layer
    # Input Tensor Shape: [batch_size, 1024]
    # Output Tensor Shape: [batch_size, 10]
    logits = tf.layers.dense(inputs=dropout, units=2)
    
    #Predict Op
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            # Generate predictions (for PREDICT and EVAL mode)
            "classes": tf.argmax(input=logits, axis=1),
            # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
            # `logging_hook`.
            "probabilities": tf.nn.softmax(logits, name="softmax_tensor"),
            "logits" : logits
            
        }
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        #Define model outputs
        # Calculate Loss (for both TRAIN and EVAL modes)
        loss = tf.losses.sparse_softmax_cross_entropy(labels=labels[:,1], logits=logits)
        
        logging_hook = tf.train.LoggingTensorHook(
            {"loss" : loss,
            },
            every_n_iter=10)
        optimizer = tf.train.AdamOptimizer(learning_rate=0.0005)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(
            mode=mode, loss=loss, train_op=train_op, training_hooks = [logging_hook])
    # Add evaluation metrics (for EVAL mode)
    if mode == tf.estimator.ModeKeys.EVAL:
        #Define model outputs
        # Calculate Loss (for both TRAIN and EVAL modes)
        loss = tf.losses.sparse_softmax_cross_entropy(labels=labels[:,1], logits=logits)
        
        predictions = {
          # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
          # `logging_hook`.
          "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
        }
        #swap this for AUROC?
        eval_metric_ops = {
          "eval_accuracy": tf.metrics.auc(
              labels=labels, predictions=predictions["probabilities"])}

        return tf.estimator.EstimatorSpec(
          mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [5]:
session_config = tf.ConfigProto(
      inter_op_parallelism_threads=32,
      intra_op_parallelism_threads=32,
      )

config = tf.estimator.RunConfig(session_config = session_config,
                                log_step_count_steps = 10,
                                save_summary_steps = 10
                               )

# Create the Estimator
car_classifier = tf.estimator.Estimator(
  model_fn=cnn_model_fn, model_dir="/home/ec2-user/convnet_model", config = config)

INFO:tensorflow:Using config: {'_model_dir': '/home/ec2-user/convnet_model', '_tf_random_seed': None, '_save_summary_steps': 10, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': intra_op_parallelism_threads: 32
inter_op_parallelism_threads: 32
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 10, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8580153e48>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [6]:
#custom input function using training_image_processor with tf.data.Dataset
def image_processor_train_input_fn(image_processor, X_train, y_train, batch_size):
    
    dataset = tf.data.Dataset.from_generator(
        generator = lambda : image_processor.flow(X_train.reshape((X_train.shape[0], 200, 200, 1)), 
                                                  y_train, batch_size, shuffle = True), 
        output_types = (np.float32, np.int32))
    
    dataset = dataset.shuffle(512).repeat()
    
    return dataset

def image_processor_eval_input_fn(X_test, y_test, batch_size):
    
    dataset = tf.data.Dataset.from_tensor_slices((X_test.astype('float32') / 255.0, 
                                                  y_test.astype('int32')))
        
    return dataset.batch(batch_size)

#Shearing, no zooming
training_image_processor = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale= 1./255,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range=15,
    horizontal_flip=True,
    data_format = 'channels_last',
)

Train

In [7]:
car_classifier.train(
      input_fn=lambda:image_processor_train_input_fn(training_image_processor, X_train, y_train, 128),
      steps=2000
    )

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensor

INFO:tensorflow:global_step/sec: 0.567988
INFO:tensorflow:loss = 0.598186, step = 1543 (17.606 sec)
INFO:tensorflow:loss = 0.598186 (17.606 sec)
INFO:tensorflow:global_step/sec: 0.572815
INFO:tensorflow:loss = 0.5377922, step = 1553 (17.458 sec)
INFO:tensorflow:loss = 0.5377922 (17.457 sec)
INFO:tensorflow:global_step/sec: 0.609635
INFO:tensorflow:loss = 0.5425193, step = 1563 (16.403 sec)
INFO:tensorflow:loss = 0.5425193 (16.404 sec)
INFO:tensorflow:global_step/sec: 0.593317
INFO:tensorflow:loss = 0.55270064, step = 1573 (16.854 sec)
INFO:tensorflow:loss = 0.55270064 (16.854 sec)
INFO:tensorflow:global_step/sec: 0.587575
INFO:tensorflow:loss = 0.57195437, step = 1583 (17.019 sec)
INFO:tensorflow:loss = 0.57195437 (17.019 sec)
INFO:tensorflow:global_step/sec: 0.574427
INFO:tensorflow:loss = 0.62030435, step = 1593 (17.408 sec)
INFO:tensorflow:loss = 0.62030435 (17.409 sec)
INFO:tensorflow:global_step/sec: 0.588256
INFO:tensorflow:loss = 0.66927475, step = 1603 (17.000 sec)
INFO:tensorf

INFO:tensorflow:loss = 0.5879687, step = 2153 (17.581 sec)
INFO:tensorflow:loss = 0.5879687 (17.581 sec)
INFO:tensorflow:global_step/sec: 0.587677
INFO:tensorflow:loss = 0.6294615, step = 2163 (17.016 sec)
INFO:tensorflow:loss = 0.6294615 (17.016 sec)
INFO:tensorflow:global_step/sec: 0.589626
INFO:tensorflow:loss = 0.57928073, step = 2173 (16.960 sec)
INFO:tensorflow:loss = 0.57928073 (16.960 sec)
INFO:tensorflow:global_step/sec: 0.569007
INFO:tensorflow:loss = 0.5789305, step = 2183 (17.574 sec)
INFO:tensorflow:loss = 0.5789305 (17.575 sec)
INFO:tensorflow:global_step/sec: 0.571659
INFO:tensorflow:loss = 0.644066, step = 2193 (17.493 sec)
INFO:tensorflow:loss = 0.644066 (17.492 sec)
INFO:tensorflow:Saving checkpoints for 2203 into /home/ec2-user/convnet_model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.585772
INFO:tensorflow:loss = 0.5661372, step = 2203 (17.071 sec)
INFO:tensorflow:loss = 0.5661372 (17.071 sec)
INFO:tensorflow:global_step/sec: 0.592306
INFO:tensorflow:loss = 0.562

INFO:tensorflow:loss = 0.5539348 (17.044 sec)
INFO:tensorflow:global_step/sec: 0.613176
INFO:tensorflow:loss = 0.5317789, step = 2703 (16.309 sec)
INFO:tensorflow:loss = 0.5317789 (16.309 sec)
INFO:tensorflow:global_step/sec: 0.567938
INFO:tensorflow:loss = 0.5558078, step = 2713 (17.608 sec)
INFO:tensorflow:loss = 0.5558078 (17.608 sec)
INFO:tensorflow:global_step/sec: 0.571737
INFO:tensorflow:loss = 0.55405676, step = 2723 (17.491 sec)
INFO:tensorflow:loss = 0.55405676 (17.491 sec)
INFO:tensorflow:global_step/sec: 0.568372
INFO:tensorflow:loss = 0.46848115, step = 2733 (17.594 sec)
INFO:tensorflow:loss = 0.46848115 (17.594 sec)
INFO:tensorflow:global_step/sec: 0.573828
INFO:tensorflow:loss = 0.55870306, step = 2743 (17.427 sec)
INFO:tensorflow:loss = 0.55870306 (17.427 sec)
INFO:tensorflow:global_step/sec: 0.567691
INFO:tensorflow:loss = 0.58713734, step = 2753 (17.615 sec)
INFO:tensorflow:loss = 0.58713734 (17.615 sec)
INFO:tensorflow:global_step/sec: 0.570417
INFO:tensorflow:loss =

INFO:tensorflow:global_step/sec: 0.567213
INFO:tensorflow:loss = 0.5243968, step = 3243 (17.630 sec)
INFO:tensorflow:loss = 0.5243968 (17.630 sec)
INFO:tensorflow:Saving checkpoints for 3252 into /home/ec2-user/convnet_model/model.ckpt.
INFO:tensorflow:Loss for final step: 0.5277533.


Evaluate Performance

In [8]:
print('training error')
eval_results = car_classifier.evaluate(
    input_fn=lambda:image_processor_eval_input_fn(X_train, y_train, 32))
print(eval_results)

print('testing error')
eval_results = car_classifier.evaluate(
    input_fn=lambda:image_processor_eval_input_fn(X_test, y_test, 32))
print(eval_results)

training error
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-21T17:57:50Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/ec2-user/convnet_model/model.ckpt-3252
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-21-17:58:19
INFO:tensorflow:Saving dict for global step 3252: eval_accuracy = 0.86665756, global_step = 3252, loss = 0.45889166
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3252: /home/ec2-user/convnet_model/model.ckpt-3252
{'eval_accuracy': 0.86665756, 'loss': 0.45889166, 'global_step': 3252}
testing error
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-21T17:58:22Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Resto

Run inference

In [10]:
predicted_classes = car_classifier.predict(
    input_fn=lambda:image_processor_eval_input_fn(X_test, y_test, 32))

testing_analysis = []

for pred, label in zip(predicted_classes, y_test):
    hold = [pred['probabilities'][1], pred['classes'], label[1], pred['logits'][0], pred['logits'][1]]
    testing_analysis.append(hold)
    
testing_analysis = pd.DataFrame(testing_analysis, columns = ['prob', 'class', 'label', 'logit_0', 'logit_1'])
testing_analysis['logit_1_decile'] = pd.qcut(testing_analysis.logit_1, 10, labels = False)
testing_analysis['logit_0_decile'] = pd.qcut(testing_analysis.logit_0, 10, labels = False)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/ec2-user/convnet_model/model.ckpt-5653
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [11]:
testing_analysis.groupby(['logit_1_decile', 'label'])['class'].value_counts()

logit_1_decile  label  class
0               0      0        103
                1      0          6
1               0      0         90
                1      0         18
2               0      0         82
                1      0         27
3               0      0         66
                1      0         42
4               0      0         28
                       1         27
                1      1         37
                       0         17
5               0      1         21
                1      1         87
6               0      1         13
                1      1         95
7               0      1         13
                1      1         96
8               0      1          8
                1      1        100
9               0      1          5
                1      1        104
Name: class, dtype: int64

In [12]:
testing_analysis.groupby(['logit_0_decile', 'label'])['class'].value_counts()

logit_0_decile  label  class
0               0      1          3
                1      1        106
1               0      1          9
                1      1         99
2               0      1         13
                1      1         96
3               0      1         12
                1      1         96
4               0      1         25
                       0          1
                1      1         81
                       0          2
5               0      0         26
                       1         25
                1      1         39
                       0         18
6               0      0         71
                1      0         35
                       1          2
7               0      0         77
                1      0         32
8               0      0         92
                1      0         16
9               0      0        102
                1      0          7
Name: class, dtype: int64

In [13]:
testing_analysis.groupby('label')['class'].value_counts()

label  class
0      0        369
       1         87
1      1        519
       0        110
Name: class, dtype: int64

In [ ]:
for i in testing_analysis.loc[testing_analysis.logit_1_decile == 9].index[0:20]:
    plt.figure()
    row = testing_analysis.iloc[i]
    plt.title('class %f label %f'%(row['class'], row['label']))
    imshow(Image.fromarray((X_test[i,:].reshape((200,200))).astype('uint8'), mode = 'L'))

SUVs and Verts

In [18]:
meta_data = pd.read_csv('/home/ec2-user/training_data_meta.csv')

#Join cleaned class data with training metadata

class_data = pd.read_csv('/home/ec2-user/stanford_labels_cleaned.csv')

meta_data = pd.merge(meta_data, class_data, on = 'class', how = 'left')

meta_data = meta_data.loc[meta_data['Body Type'].isin(['SUV', 'Convertible'
                                                      ])].copy()

meta_data['is_sedan_target'] = (meta_data['Body Type'] == 'SUV').astype(int)

image_dict = p.load(open('/home/ec2-user/scaled_bounded_grayscale_dict.p', 'rb'))

oot_data = []
for i in meta_data[['is_sedan_target', 'fname']].iterrows():
    row = [i[1]['is_sedan_target']]
    row.extend(image_dict[i[1]['fname']].flatten())
    oot_data.append(row)

oot_data = np.array(oot_data).astype(float)

In [19]:
X_oot = oot_data[:,1:]

y_oot = oot_data[:,0].astype('int32')
y_oot = np.concatenate([1 - y_oot.reshape(-1,1), y_oot.reshape(-1,1)], axis = 1)

In [20]:
print('oot error')
eval_results = car_classifier.evaluate(
    input_fn=lambda:image_processor_eval_input_fn(X_oot, y_oot, 32))
print(eval_results)

oot error
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-21T15:51:47Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/ec2-user/convnet_model/model.ckpt-5653
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-21-15:52:17
INFO:tensorflow:Saving dict for global step 5653: eval_accuracy = 0.82551754, global_step = 5653, loss = 0.80654323
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5653: /home/ec2-user/convnet_model/model.ckpt-5653
{'eval_accuracy': 0.82551754, 'loss': 0.80654323, 'global_step': 5653}


In [23]:
predicted_classes = car_classifier.predict(
    input_fn=lambda:image_processor_eval_input_fn(X_oot, y_oot, 32))

testing_analysis = []

for pred, label in zip(predicted_classes, y_oot):
    hold = [pred['probabilities'][1], pred['classes'], label[1], pred['logits'][0], pred['logits'][1]]
    testing_analysis.append(hold)
    
testing_analysis = pd.DataFrame(testing_analysis, columns = ['prob', 'class', 'label', 'logit_0', 'logit_1'])
testing_analysis['logit_1_decile'] = pd.qcut(testing_analysis.logit_1, 10, labels = False)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/ec2-user/convnet_model/model.ckpt-5653
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [24]:
testing_analysis.groupby(['logit_1_decile', 'label'])['class'].value_counts()

logit_1_decile  label  class
0               0      0        231
                1      0         29
1               0      0        196
                1      0         63
2               0      0        160
                1      0         97
                       1          2
3               0      1         82
                       0         40
                1      1        104
                       0         34
4               0      1        113
                1      1        146
5               0      1         68
                1      1        191
6               0      1         49
                1      1        211
7               0      1         40
                1      1        219
8               0      1         30
                1      1        229
9               0      1         27
                1      1        233
Name: class, dtype: int64

In [25]:
testing_analysis.groupby('label')['class'].value_counts()

label  class
0      0         627
       1         409
1      1        1335
       0         223
Name: class, dtype: int64

In [ ]:
for i in testing_analysis.loc[testing_analysis.logit_1_decile == 3].index[0:20]:
    plt.figure()
    row = testing_analysis.iloc[i]
    plt.title('class %f label %f'%(row['class'], row['label']))
    imshow(Image.fromarray((X_oot[i,:].reshape((200,200))).astype('uint8'), mode = 'L'))

Train and Evaluate

In [9]:
train_spec = tf.estimator.TrainSpec(
     input_fn=lambda:image_processor_train_input_fn(
         training_image_processor, X_train, y_train, 32), 
     max_steps=4100)


eval_spec = tf.estimator.EvalSpec(
    input_fn=lambda:image_processor_eval_input_fn(X_train, y_train, 32)
)

In [11]:
evaluate_results = tf.estimator.train_and_evaluate(car_classifier, train_spec, eval_spec)

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Skipping training since max_steps has already saved.


Export to SavedModel

In [24]:
def serving_input_receiver_fn():
    inputs = {
        'input' : tf.placeholder(tf.float32, [None, 200, 200, 1]),
    }
    
    inputs['input'] = tf.divide(inputs['input'], 255.0)
    return tf.estimator.export.ServingInputReceiver(inputs, inputs)


In [26]:
car_classifier.export_savedmodel('/home/ec2-user/convnet_saved_model/', serving_input_receiver_fn,
                            strip_default_attrs=True)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from /home/ec2-user/convnet_model/model.ckpt-4100
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /home/ec2-user/convnet_saved_model/temp-b'1553120111'/saved_model.pb


b'/home/ec2-user/convnet_saved_model/1553120111'